In [15]:
#data loading
import dask.dataframe as dd

dtype = {'trusted_indicator': 'object'}
global_data = dd.read_csv(r'C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Transfers/Datasets/0globaldata.csv',
                          dtype = dtype)
                          
                          

In [16]:
global_data = global_data[['canal', 'operativa', 'entity', 'reference', 'trusted_indicator', 'iban_orig', 'iban_dest', 
                           'amount', 'accountbalance', 'ipaddress', 'is_fraud', 'is_tablet', 'is_pc', 'is_mobile', 'is_touch', 
                           'clientid', 'browser_family', 'os_family', 'hour', 'week', 'timestamp']]

In [17]:
#make sure the columns were well selected
global_data.shape

(Delayed('int-17128f17-dcd4-4e87-b601-f13bc5331da4'), 21)

In [19]:
#total lines in the csv
len(global_data)

29051438

In [2]:
#date in which the data ends
max(global_data['date'])

20200531

In [3]:
#date in which the data begins
min(global_data['date'])

20190801

In [77]:
#how many frauds in the global dataset
target_col = 'is_fraud'
global_data[(global_data[target_col] == 1)].count().compute()

Unnamed: 0               821
timestamp                821
ID                       821
rowid                    821
canal                    821
                        ... 
amount_categories        821
cons_freq_fingerprint    821
cons_freq_clientid       821
cons_freq_ipaddress      821
cons_time_fingerprint    821
Length: 93, dtype: int64

In [2]:
#there's no ocurrencies of bots in the entire dataset so it can be eliminated - column with only zeros
global_data["is_bot"].value_counts().compute()

False    29051438
Name: is_bot, dtype: int64

In [4]:
#find if there is NaN values
global_data.isna().sum(axis = 0).compute()

Unnamed: 0                           0
timestamp                            0
ID                                   0
rowid                                0
canal                                0
operativa                            0
client                               0
clientid                             0
entity                               0
reference                            0
trusted_indicator                    0
iban_orig                            0
iban_dest                            0
amount                               0
accountbalance                       0
fingerprint                          0
ipaddress                            0
description_originator               0
description_beneficiary              0
is_fraud                             0
dummy_var                            0
is_mobile                            0
is_tablet                            0
is_pc                                0
is_touch                             0
is_bot                   

In [5]:
#find if there is missing values
global_data.isnull().sum().compute()

Unnamed: 0                           0
timestamp                            0
ID                                   0
rowid                                0
canal                                0
operativa                            0
client                               0
clientid                             0
entity                               0
reference                            0
trusted_indicator                    0
iban_orig                            0
iban_dest                            0
amount                               0
accountbalance                       0
fingerprint                          0
ipaddress                            0
description_originator               0
description_beneficiary              0
is_fraud                             0
dummy_var                            0
is_mobile                            0
is_tablet                            0
is_pc                                0
is_touch                             0
is_bot                   

In [12]:
#values that operativa takes
set(global_data['operativa'])

{'APGCMP',
 'APGEST',
 'APGINT',
 'APGSEG',
 'APGSRV',
 'APGTEL',
 'APGTRP',
 'ATFINT',
 'ATFIPS',
 'ATFITC',
 'ATFITI',
 'ATFSEP',
 'PAGCMP',
 'PAGEST',
 'PAGINT',
 'PAGSEC',
 'PAGSEG',
 'PAGSRV',
 'PAGTEL',
 'PAGTRP',
 'RTFINT',
 'RTFIPS',
 'RTFITC',
 'RTFSEP',
 'TRFIBS',
 'TRFINT',
 'TRFIPS',
 'TRFITC',
 'TRFITI',
 'TRFMBW',
 'TRFSEP'}

In [20]:
#select all operativas that refer to transfers 
target_col = 'operativa'

transfers = global_data[(global_data[target_col] == 'TRFIBS') | (global_data[target_col] == 'TRFINT') | 
                        (global_data[target_col] == 'TRFIPS') | (global_data[target_col] == 'TRFITC') |
                        (global_data[target_col] == 'TRFITI') | (global_data[target_col] == 'TRFMBW') |
                        (global_data[target_col] == 'TRFSEP')].copy()

In [21]:
#number of rows in the transfers csv
len(transfers)

15648273

In [22]:
#number of frauds in transfers
target_col = 'is_fraud'
transfers[(transfers[target_col] == 1)].count().compute()

canal                569
operativa            569
entity               569
reference            569
trusted_indicator    569
iban_orig            569
iban_dest            569
amount               569
accountbalance       569
ipaddress            569
is_fraud             569
is_tablet            569
is_pc                569
is_mobile            569
is_touch             569
clientid             569
browser_family       569
os_family            569
hour                 569
week                 569
timestamp            569
dtype: int64

In [23]:
#save all transfers data in a csv file
transfers.to_csv(r'C:\Users\BeatrizCarvalho\0transfers.csv', index = False, single_file = True)

['C:/Users/BeatrizCarvalho/0transfers.csv']

In [ ]:
#divide in frauds and nonfrauds so that repeated fraud data won't be eliminated

In [30]:
#select only the frauds
target_col = 'is_fraud'
frauds = transfers[(transfers[target_col] == 1)].copy()

In [33]:
#select only the non-frauds 
nonfrauds = transfers[(transfers[target_col] == 0)].copy()

In [34]:
#number of rows in nonfrauds data before the duplicates removal
len(nonfrauds)

15647704

In [35]:
#drop duplicates
nonfrauds_nd = nonfrauds.drop_duplicates()

In [36]:
#number of rows in nonfrauds after the duplicates removal (before there was 15647704)
len(nonfrauds_nd)

15398238

In [37]:
#number of duplicates
15647704 - 15398238

249466

In [38]:
#merge the dataframes again
transfers_nd = nonfrauds_nd.append(frauds)

In [39]:
#make sure the merge was done correctly
len(transfers_nd)

15398807

In [40]:
#save dataframe without the duplicates in a single csv file
transfers_nd.to_csv(r'C:\Users\BeatrizCarvalho\0transfers_nd.csv', index = False, single_file = True)

['C:/Users/BeatrizCarvalho/0transfers_nd.csv']